# Sudoku Using a MIP Solver

Solve sudoku puzzles using a MIP (Mixed Integer linear Program) solver. We've integrated three solvers, HiGHS, Gurobi, and GLPK. Gurobi requires a license to run so is only used in the final solve. If you don't have
a Gurobi license, that run will produce an error.

## Define a function to map a digit character to its value.

We define a function that maps from a Unicode character to its "value" from the
perspective of Sudoku. We want to support traditional Sudoku (of rank 3) as
well as higher ranks, up to rank 6, so we need a total of 36 symbols. Traditional
Sudoku uses `"123456789"`. To get 36 symbols, we'll also use `0` and `A`
through `Z`, so our symbols are `"1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ"`.

This function should map a Unicode character to its position in our symbol list,
and return `-1` if the Unicode character is not one of our symbols.

In [1]:
// Accepts a Unicode code point. Returns the index into
// "1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ", or -1 if not found.
// Note that 0 follows 9.

func ToDigit(ch) :=
  ch - "1"[0] if "1"[0] <= ch <= "9"[0]      else
  9           if           ch  = "0"[0]      else
  ch - "A"[0] + 10 if "A"[0] <= ch <= "Z"[0] else
  ch - "A"[0] + 10 if "A"[0] <= ch <= "Z"[0] else
  -1;

## Define the module.

The parameter `M` specifies the grid size value, with default of `3`. This can handle `M` values of `3`, `4`, `5`, and `6`.

In [2]:
Sudoku := module {
    param M := 3;
    const N := M * M;
    const NumCells := N * N;
    const NumMoves := NumCells * N;

    // All possible "moves" where "move" means "place a value in a cell".
    const Moves := Range(NumMoves)
        ->ForEach(as Id,
            With(cell: Id div N, row: cell div N, col: cell mod N,
                { Id, XRow: row, XCol: col, YBlk: row div M * M + col div M, ZVal: Id mod N }));

    // Group the moves in various ways. These are used for constraints below.
    // Each group of these represent, respectively:
    // * The possible values for a particular cell.
    // * The possible placements of a particular value in a particular row.
    // * The possible placements of a particular value in a particular column.
    // * The possible placements of a particular value in a particular block.
    const MovesByRowCol := Moves->GroupBy([key] _:XRow, [key] _:XCol);
    const MovesByValRow := Moves->GroupBy([key] _:ZVal, [key] _:XRow);
    const MovesByValCol := Moves->GroupBy([key] _:ZVal, [key] _:XCol);
    const MovesByValBlk := Moves->GroupBy([key] _:ZVal, [key] _:YBlk);

    // The fixed moves defined by a particular puzzle.
    // Note that this is a parameter, not constant, so can be set externally via
    // module projection (see below).
    param Fixed :=
        "    2  7 " &
        "    34   " &
        "358      " &
        "5 48     " &
        "   1   89" &
        "  2     6" &
        "24    7  " &
        " 9   52  " &
        "    671  ";

    // Map from the Fixed text value to the required moves.
    const ImposedIds :=
        Range(NumCells)
        ->{ cell: it, value: Fixed[it]->ToDigit() }
        ->TakeIf(0 <= value < N)
        ->Map(N * cell + value);
    const NumImposed := ImposedIds->Count();
    const ImposedFlags := Tensor.From(Range(NumMoves)->(it in ImposedIds));

    // Define a variable (tensor of bool) for the moves that have been made.
    var Flags from Tensor.Fill(false, NumMoves) def ImposedFlags;

    // Define a measure for the number of moves made.
    // Need to maximize this without violating constraints.
    msr NumMade := Flags.Values->Sum();

    // Define the constraints.

    // Require the imposed moves.
    con NeedImposed := ForEach(id:ImposedIds, Flags[id] = 1);

    // Require no "duplicates". That is, for each MovesByXxxYyy above, there
    // can be at most one move from each group.
    // NOTE: we use <= 1 rather than = 1 so we can "do the best possible" when
    // there is no complete solution. See below for examples.
    con OnePerRowCol := ForEach(c:MovesByRowCol, Sum(c, Flags[Id]) <= 1);
    con OnePerValRow := ForEach(c:MovesByValRow, Sum(c, Flags[Id]) <= 1);
    con OnePerValCol := ForEach(c:MovesByValCol, Sum(c, Flags[Id]) <= 1);
    con OnePerValBlk := ForEach(c:MovesByValBlk, Sum(c, Flags[Id]) <= 1);

    // The Board is for "pretty" display of the result.
    const Symbols := "_1234567890ABCDEFGHIJKLMNOPQRSTUVWXYZ";
    let Board :=
        Moves
        ->TakeIf(Flags[Id])
        ->SortUp(XRow, XCol)
        ->GroupBy(_:XRow)
        ->ForEach(With(row: it,
            ForEach(c: Range(N), With(i: First(row, XCol = c).ZVal + 1 ?? 0, Symbols[i:*1])))
            ->Text.Concat("|"))
        ->Text.Concat("\n");
};

Display the initial board with only the `Fixed` cells (required values) filled in.

In [3]:
Sudoku.Board;
Sudoku.NumMade;

_|_|_|_|2|_|_|7|_
_|_|_|_|3|4|_|_|_
3|5|8|_|_|_|_|_|_
5|_|4|8|_|_|_|_|_
_|_|_|1|_|_|_|8|9
_|_|2|_|_|_|_|_|6
2|4|_|_|_|_|7|_|_
_|9|_|_|_|5|2|_|_
_|_|_|_|6|7|1|_|_
24


Show the constraint values. If any are `false`, then the `Fixed` cells violate the constraints.

In [4]:
Sudoku.NeedImposed->All();
Sudoku.OnePerRowCol->All();
Sudoku.OnePerValRow->All();
Sudoku.OnePerValCol->All();
Sudoku.OnePerValBlk->All();
Sudoku.NumMade;

true
true
true
true
true
24


## Solve It

In [5]:
#!time
Sln := Sudoku->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
4|6|1|5|2|8|9|7|3
7|2|9|6|3|4|8|5|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|4|7|3|5|1|6
2|4|6|9|8|1|7|3|5
1|9|7|3|4|5|2|6|8
8|3|5|2|6|7|1|9|4
81


Wall time: 216.5431ms

In [6]:
#!time
Sln2 := Sudoku->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
4|6|1|5|2|8|9|7|3
7|2|9|6|3|4|8|5|1
3|5|8|7|1|9|6|4|2
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|4|7|3|5|1|6
2|4|6|9|8|1|7|3|5
1|9|7|3|4|5|2|6|8
8|3|5|2|6|7|1|9|4
81


Wall time: 86.8288ms

In [7]:
Sln.Board = Sln2.Board

true


## Partial Solve

This examples starts with the same required values, but with "2" added at the end of the 2nd row,
which is inconsistent with the solution. The result is the board cannot be completely filled in.
According to the solvers, the maximum number of cells that can be filled in is 79, leaving 2 unfilled.

Note that the two solvers find different configurations that achieve the maximum of 79 cells filled.

In [8]:
Bad := Sudoku=>{
    Fixed:
        "    2  7 " &
        "    34  2" &
        "358      " &
        "5 48     " &
        "   1   89" &
        "  2     6" &
        "24    7  " &
        " 9   52  " &
        "    671  "
};

Bad.Board;
Bad.NumMade;

_|_|_|_|2|_|_|7|_
_|_|_|_|3|4|_|_|2
3|5|8|_|_|_|_|_|_
5|_|4|8|_|_|_|_|_
_|_|_|1|_|_|_|8|9
_|_|2|_|_|_|_|_|6
2|4|_|_|_|_|7|_|_
_|9|_|_|_|5|2|_|_
_|_|_|_|6|7|1|_|_
25


In [9]:
#!time
Sln := Bad->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
4|_|6|5|2|1|9|7|3
1|7|9|6|3|4|8|5|2
3|5|8|9|7|_|6|4|1
5|1|4|8|9|6|3|2|7
7|6|3|1|4|2|5|8|9
9|8|2|7|5|3|4|1|6
2|4|1|3|8|9|7|6|5
6|9|7|4|1|5|2|3|8
8|3|5|2|6|7|1|9|4
79


Wall time: 74.3984ms

In [10]:
#!time
Sln2 := Bad->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
4|6|1|5|2|8|9|7|3
7|_|9|6|3|4|8|5|2
3|5|8|7|1|9|6|4|_
5|1|4|8|9|6|3|2|7
6|7|3|1|5|2|4|8|9
9|8|2|4|7|3|5|1|6
2|4|6|9|8|1|7|3|5
1|9|7|3|4|5|2|6|8
8|3|5|2|6|7|1|9|4
79


Wall time: 83.5172ms

In [11]:
Sln.Board = Sln2.Board

false


## Hardest Sudoku

The "internet" claims this is the world's most difficult sudoku puzzle, taken from
[here](https://www.kristanix.com/sudokuepic/worlds-hardest-sudoku.php). The author of it,
Finnish mathematician Arto Inkala, calls it AI Escargot.

In [12]:
Hardest := Sudoku=>{
    Fixed:
        "1    7 9 " &
        " 3  2   8" &
        "  96  5  " &
        "  53  9  " &
        " 1  8   2" &
        "6    4   " &
        "3      1 " &
        " 4      7" &
        "  7   3  "
};

Hardest.NumImposed;

23


In [13]:
#!time
Sln := Hardest->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
1|6|2|8|5|7|4|9|3
5|3|4|1|2|9|6|7|8
7|8|9|6|4|3|5|2|1
4|7|5|3|1|2|9|8|6
9|1|3|5|8|6|7|4|2
6|2|8|7|9|4|1|3|5
3|5|6|4|7|8|2|1|9
2|4|1|9|3|5|8|6|7
8|9|7|2|6|1|3|5|4
81


Wall time: 2252.5438ms

In [14]:
#!time
Sln2 := Hardest->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
1|6|2|8|5|7|4|9|3
5|3|4|1|2|9|6|7|8
7|8|9|6|4|3|5|2|1
4|7|5|3|1|2|9|8|6
9|1|3|5|8|6|7|4|2
6|2|8|7|9|4|1|3|5
3|5|6|4|7|8|2|1|9
2|4|1|9|3|5|8|6|7
8|9|7|2|6|1|3|5|4
81


Wall time: 190.7778ms

In [15]:
Sln.Board = Sln2.Board

true


## Another difficult one

This is another difficult one, also authored by Arto Inkala.

In [16]:
Finnish := Sudoku=>{
    Fixed:
        "8        " &
        "  36     " &
        " 7  9 2  " &
        " 5   7   " &
        "    457  " &
        "   1   3 " &
        "  1    68" &
        "  85   1 " &
        " 9    4  "
};

In [17]:
#!time
Sln := Finnish->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
8|1|2|7|5|3|6|4|9
9|4|3|6|8|2|1|7|5
6|7|5|4|9|1|2|8|3
1|5|4|2|3|7|8|9|6
3|6|9|8|4|5|7|2|1
2|8|7|1|6|9|5|3|4
5|2|1|9|7|4|3|6|8
4|3|8|5|2|6|9|1|7
7|9|6|3|1|8|4|5|2
81


Wall time: 1307.9158ms

In [18]:
#!time
Sln2 := Finnish->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
8|1|2|7|5|3|6|4|9
9|4|3|6|8|2|1|7|5
6|7|5|4|9|1|2|8|3
1|5|4|2|3|7|8|9|6
3|6|9|8|4|5|7|2|1
2|8|7|1|6|9|5|3|4
5|2|1|9|7|4|3|6|8
4|3|8|5|2|6|9|1|7
7|9|6|3|1|8|4|5|2
81


Wall time: 205.5495ms

In [19]:
Sln.Board = Sln2.Board

true


## Generate a Sudoku board from scratch

Here we specify only the first row. The solution isn't unique and indeed the two solvers produce different results. 

In [20]:
#!time
Sln := Sudoku=>{ Fixed: "123456789" }->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
1|2|3|4|5|6|7|8|9
8|7|5|9|1|3|6|4|2
9|6|4|2|8|7|1|5|3
2|9|1|6|4|8|5|3|7
7|4|6|3|2|5|8|9|1
5|3|8|1|7|9|4|2|6
6|5|2|8|9|1|3|7|4
4|1|7|5|3|2|9|6|8
3|8|9|7|6|4|2|1|5
81


Wall time: 966.1083ms

In [21]:
#!time
Sln2 := Sudoku=>{ Fixed: "123456789" }->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
1|2|3|4|5|6|7|8|9
8|7|4|9|1|3|2|5|6
9|5|6|8|7|2|3|1|4
5|8|9|1|6|7|4|2|3
6|4|2|5|3|9|1|7|8
3|1|7|2|4|8|6|9|5
2|9|1|3|8|4|5|6|7
4|6|8|7|2|5|9|3|1
7|3|5|6|9|1|8|4|2
81


Wall time: 383.3877ms

In [22]:
Sln.Board = Sln2.Board;

false


## Produce a rank 4 Sudoku board

Generate a board (specifying the first row) of rank 4.

In [23]:
#!time
Sln := Sudoku=>{ Fixed: "1234567890ABCDEF", M: 4 }->Maximize(NumMade, "highs");
Sln.Board;
Sln.NumMade;

Solver: HiGHS
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F
5|7|E|0|C|F|1|9|2|D|3|8|4|6|B|A
A|B|8|6|D|0|4|E|1|C|7|F|3|5|2|9
F|C|D|9|2|3|A|B|6|4|5|E|7|8|1|0
E|8|1|C|A|9|6|F|0|2|D|7|5|3|4|B
4|A|F|B|3|D|8|5|E|1|6|C|0|7|9|2
2|0|6|3|E|7|B|C|4|8|9|5|D|F|A|1
D|9|5|7|4|2|0|1|F|A|B|3|8|E|C|6
8|1|0|5|7|A|9|4|B|F|E|D|6|2|3|C
9|3|7|F|8|B|E|2|5|6|C|A|1|0|D|4
B|D|4|E|6|1|C|3|8|9|2|0|F|A|5|7
C|6|2|A|F|5|D|0|7|3|1|4|9|B|8|E
0|4|B|8|1|C|3|7|D|E|F|2|A|9|6|5
6|E|C|2|9|8|5|D|A|7|0|1|B|4|F|3
3|5|A|D|0|E|F|6|C|B|4|9|2|1|7|8
7|F|9|1|B|4|2|A|3|5|8|6|E|C|0|D
256


Wall time: 19754.3975ms

In [24]:
#!time
Sln2 := Sudoku=>{ Fixed: "1234567890ABCDEF", M: 4 }->Maximize(NumMade, "glpk");
Sln2.Board;
Sln2.NumMade;

Solver: GLPK
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F
5|6|7|8|1|2|3|4|C|D|E|F|9|0|A|B
9|0|A|B|C|D|E|F|1|2|3|4|5|6|7|8
C|D|E|F|9|0|A|B|5|6|7|8|1|2|3|4
2|1|4|3|6|5|8|7|0|9|B|A|D|C|F|E
6|5|8|7|2|1|4|3|D|C|F|E|0|9|B|A
0|9|B|A|D|C|F|E|2|1|4|3|6|5|8|7
D|C|F|E|0|9|B|A|6|5|8|7|2|1|4|3
3|4|1|2|7|8|5|6|A|B|9|0|E|F|C|D
7|8|5|6|3|4|1|2|E|F|C|D|A|B|9|0
A|B|9|0|E|F|C|D|3|4|1|2|7|8|5|6
E|F|C|D|A|B|9|0|7|8|5|6|3|4|1|2
4|3|2|1|8|7|6|5|B|A|0|9|F|E|D|C
8|7|6|5|4|3|2|1|F|E|D|C|B|A|0|9
B|A|0|9|F|E|D|C|4|3|2|1|8|7|6|5
F|E|D|C|B|A|0|9|8|7|6|5|4|3|2|1
256


Wall time: 362.4651ms

In [25]:
Sln.Board = Sln2.Board;

false


## Produce a rank 5 Sudoku board

Now try rank 5, with Gurobi. HiGHS and GLPK don't do well on this one.
Gurobi even struggles with it. Note that this is severely under constrained.

In [26]:
#!time
Sln := Sudoku=>{ Fixed: "1234567890ABCDEFGHIJKLMNO", M: 5 }->Maximize(NumMade, "gurobi");
Sln.Board;
Sln.NumMade;

Solver: Gurobi
1|2|3|4|5|6|7|8|9|0|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O
6|L|E|8|K|J|C|M|A|F|H|4|0|I|3|7|9|N|5|O|D|1|B|G|2
O|9|M|0|J|G|H|E|B|2|F|8|N|K|7|4|D|A|1|L|3|5|6|C|I
F|A|G|N|C|I|4|D|K|L|5|1|6|M|O|B|3|8|E|2|H|7|9|0|J
B|D|H|7|I|N|1|3|O|5|G|L|9|J|2|0|M|K|6|C|F|4|8|A|E
G|F|L|B|A|K|8|6|3|1|0|D|7|O|N|9|C|4|M|E|J|I|H|2|5
2|3|9|M|1|C|I|B|G|7|J|E|5|6|8|O|H|D|0|N|4|F|L|K|A
8|I|6|C|7|F|E|H|0|O|2|3|G|L|4|A|K|B|J|5|9|M|N|1|D
N|4|0|E|O|L|M|5|J|D|K|H|A|9|1|3|7|I|2|F|8|G|C|B|6
D|5|K|J|H|4|N|9|2|A|C|M|B|F|I|6|1|L|8|G|0|3|E|O|7
H|E|1|G|F|O|0|7|C|I|8|J|D|A|L|2|B|6|4|9|M|N|5|3|K
K|7|B|6|8|M|2|1|L|N|I|G|H|4|0|E|5|O|F|3|C|D|A|J|9
A|M|5|2|0|B|D|J|4|E|9|6|1|3|K|L|N|C|H|I|7|O|F|8|G
C|O|J|I|3|A|F|K|8|9|7|N|E|5|B|M|0|1|G|D|2|6|4|H|L
L|N|D|9|4|H|6|G|5|3|M|O|F|2|C|8|A|J|7|K|I|B|0|E|1
4|0|2|1|D|8|J|N|I|B|L|5|M|7|G|K|E|9|O|6|A|C|3|F|H
9|C|7|3|M|D|L|0|1|K|E|I|2|H|6|J|F|G|B|A|N|8|O|5|4
J|8|O|L|6|2|5|F|M|C|1|9|K|B|A|D|4|3|N|H|G|E|7|I|0
5|K|I|F|G|7|3|A|E|H|4|0|O|N|D|1|2|M|C|8|6|9|J|L|B
E|B|A|H|N|9|O|4|6|G|3|C|J|8|F|5|I|0

Wall time: 63867.2102ms